## Юнит 8. Введение в нейронные сети
### Skillfactory: DSPR-19
### DL-2. Практика с основными фреймворками 

### 1. Уровни абстракции

Самое время приступать к практике с основными фреймворками! В этом модуле мы научимся работать с двумя — **TensorFlow** и **PyTorch.**

Но для начала разберёмся, насколько глубоко мы вообще можем погружаться в обучение нейросетей.

Существует несколько уровней абстракции, на которых мы можем сконструировать нейросеть:

- **Нижний уровень (NumPy)** — нет готовых высокоуровневых блоков, нет оптимизаторов, отсутствует возможность параллельных расчётов и использования GPU. Используется в учебных целях для иллюстрации принципов работы нейросетей.
- **Фреймворки автоматического дифференцирования (PyTorch и TensorFlow)** — вы определяете вычислительный граф и некоторые гиперпараметры, а всё остальное фреймворк делает за вас. Позволяет строить нейросети произвольной сложности.
- **Фреймворки готовых моделей (Transformers и DeepPavlov)** — скрывают технические детали самой нейросети и представляют высокоуровневый интерфейс для основных операций: обучения и инференса. Являются обёртками для больших и сложных нейросетей.

### ФРЕЙМВОРКИ АВТОМАТИЧЕСКОГО ДИФФЕРЕНЦИРОВАНИЯ 

Рассмотрим, какие фреймворки представлены на рынке.



### 2. Вспоминаем линейную классификацию

Прежде чем знакомиться с фреймворками, давайте снова поговорим о том, что такое линейная классификация. Это нам обязательно пригодится далее.

### ЛИНЕЙНАЯ КЛАССИФИКАЦИЯ 

Представьте, что у нас есть признаки x = (x1, x2) и есть выборка положительных и отрицательных точек y ∈ {+1, −1}

Нам нужно найти разделяющую гиперплоскость между ними. В данном случае это просто линия. Линия задаётся тремя коэффициентами:

Нам нужно найти три коэффициента w, которые зададут линию. Далее мы можем взять точку х и понять, где она находится относительно линии: выше или ниже. Для этого нам нужно узнать знак линейной комбинации. Вектор весов w задаёт нормаль к нашей линии, то есть он перпендикулярен ей (фиолетовый вектор на графике ниже).

Линейная комбинация — это скалярное произведение и длина проекции какого-нибудь другого вектора на наш вектор w.

Поэтому проекция становится разных знаков. Из этих соображений мы делаем линейный классификатор. Наш алгоритм: 

Здесь появляется знак нашей линейной комбинации. Настроить линейный классификатор — значит найти эти коэффициенты. 

### ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ 

Она тоже решает задачу классификации, но в конце применяется не функция знака, а сигмоидная функция. Она превращает длину проекции в уверенность.

Уверенность и неуверенность появляется из-за краевых эффектов: на границе классов может быть какой-то шум, и в классификации точек, которые находятся рядом с красной разделяющей линией, мы не очень уверены. 

А если мы уходим далеко от линии вглубь классов, то предполагается, что мы более уверены в этом предсказании. Сигмоида делает именно это — превращает длину проекции линейной комбинации в уверенность.

Сигмоида устроена не случайным образом:

Если длина проекции 0 (точка ровно на красной линии) , то сигмоида даёт . Логистическая регрессия предсказывает вероятность положительного класса. Вероятность отрицательного будет единица минус предсказанная вероятность положительного класса. 

#### ДРУГОЙ ПРИМЕР 

Представим, что у нас есть следующая задача:

Чтобы решить подобную задачу, мы можем «подпереть» треугольник тремя линиями и сделать алгоритм, используя только логистическую регрессию.

Для начала мы отделим минусы слева и построим логистическую регрессию:

Эти данные подадим для обучения логистической регрессии и получим коэффициенты красной линии. 

Отделим остальные минусы:

Все коэффициенты мы получили практически вручную. Сейчас у нас есть коэффициенты трёх логистических регрессий, каждая из которых решает свою маленькую подзадачу. 

Теперь возьмём какую-нибудь точку и посмотрим, какие три предсказания дают эти линии в точке:

### ЧТО ДЕЛАТЬ С ЭТИМИ ТРЕМЯ ЗНАЧЕНИЯМИ? 

В координатах х1 и х2 эта задача не решается. Поэтому полученные коэффициенты мы можем рассматривать как новые координаты. 

Получим три признака, каждый из которых говорит, где мы находимся относительно каждой стороны треугольника. 

Давайте возьмём наш целевой признак y, добавим его в нашу новую таблицу, где наши новые признаки с предсказаниями, и попробуем решить её с новыми признаками с помощью линейной логистической регрессии. На новой выборке получим логистическую регрессию:

Теперь она даёт нам некоторые коэффициенты и взвешивает уже не признаки, а предсказания. То, что мы получили — простейшая нейросеть.

### Задание 2.1
Почему в логистической регрессии используется Сигмоида, а не Знак (sign)?  

Ответ: Сигмоида плавная и непрерывная, что позволяет её дифференцировать верно

### 3. Граф вычислений


На данный момент мы «руками» нашли параметры всех линий:

При этом на нашу сложную комбинацию функций можно посмотреть как на граф вычислений. У графа есть вершины, и каждая вершина — вычисляемое значение.

Также есть рёбра — зависимости, которые имеют направления (на картинке ниже это стрелочки). Ребро идёт от х1 к z1 в случае, если нам необходим х1, чтобы вычислить значение z1. Это граф зависимости между вычисляемыми значениями:

Граф соответствует комбинации наших функций. Такой граф называют многослойным персептроном, и здесь уже можно видеть некоторые слои:

- входной слой (признаки);
- скрытый слой (нейроны);
- выходной слой (предсказания).

### НЕЛИНЕЙНОСТИ В НЕЙРОНАХ 

Возьмём в качестве примера следующий граф:

Если нелинейности убрать, то на этом примере видно, что наша модель станет очень простой: мы можем подставить выражения для z1 и z2 в нашу модель а:

Мы можем раскрыть скобки, привести подобные слагаемые и всё, что мы получим — линейную комбинацию х1 и х2. При этом модель сложнее не становится.



### Задание 3.1
Зачем перцептроны делают многослойными?  

Ответ: Несколько слоёв позволяют разделить данные, которые не делятся одной линией 

### Задание 3.2
Что будет, если убрать нелинейность из скрытого слоя?  


Ответ:  
- Скрытого слоя как бы не станет
- Модель не сможет работать со сложными данными, не делящимися одной линией


### 4. MLP

**MLP** — это простейший пример нейросети:

Слои в MLP называют:

- Dense layer (плотный);
- Fully-connected layer (полносвязный).

### Архитектура MLP:

- количество слоёв;
- количество нейронов в каждом слое;
- функция активации, которую будем использовать.


### Задание 4.1

Какие из параметров обучаемые, а какие являются гиперпараметрами нейросети?

Параметр | Вид
:-- | :--
__Веса связей между входными нейронами и слоем Z__ |	Обучаемые параметры 
__Количество нейронов в слое Z__ |	гиперпараметры
__Функция активации в слое Z__ |	гиперпараметры
__Веса связей между слоем Z и слоем H__ |	Обучаемые параметры

### 5. TensorFlow

**TENSORFLOW (TF) — DEEP LEARNING ФРЕЙМВОРК**

Основа вычислений в **TF — граф**. Каждая вершина графа — это одна операция, у которой есть входы и выходы.

- Вход любой операции — это набор тензоров (многомерных массивов).
- Выход любой операции — это тоже набор тензоров.  

А у нас целый граф операций, между которыми перекидываются тензоры.

#### Дополнительные материалы:
https://www.oreilly.com/content/hello-tensorflow/


### Задание 5.1
Тензором являются:
- данные, поступающие на вход
- веса связей между слоями
- ошибка предсказания